<a href="https://colab.research.google.com/github/Peres-vinicius/collecting_data_twitter/blob/main/collecting_twitter_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweepy==4.6

## IMPORTS

In [1]:
import tweepy as tw
import time as tm
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

## FUNCTIONS

In [2]:
def create_db(name):
    # cria o bd, a table registros e retorna o con e o cur
    name = name + '.db'
    con = sqlite3.connect(name)
    cur = con.cursor()
    cur.execute('CREATE TABLE registros (texto TEXT)')
    return con, cur

In [3]:
def connect_twitter(BEARER_TOKEN, CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET):
    # conectando ao twitter
    return tw.Client(BEARER_TOKEN, CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    

In [4]:
def extracting_data(client_twitter, query, start_time, end_minutes_time):
    """
    client_twitter : obj retornado da função connect_twitter
    query : termo para busca
    start_time : obj do tipo datetime
    end_time : int com minutos
    """
    result = []
    minutes = 1
    while minutes <= end_minutes_time:
        end_time = start_time + timedelta(days = 0, minutes = minutes)
        print(end_time)
        res = client_twitter.search_recent_tweets(query=query, max_results=100, start_time=start_time, end_time=end_time)
        res = res.data
        result.append(res)
        tm.sleep(5)
        minutes +=1
    return result

In [5]:
def insert_db(cur, con, data):
    for i in data:
        for x in i:
            cur.execute("INSERT INTO registros (texto) VALUES (?);", (x.text,))
        
    con.commit()
    con.close()

In [6]:
def read_data(name_db):
    # conectando ao banco sqlite
    db = name_db + '.db'
    con = sqlite3.connect(db)
    cur = con.cursor()
    # busca todos os registros
    cur.execute('SELECT * FROM registros')
    res = cur.fetchall()
    con.close()
    #adiciona a um df
    res = pd.DataFrame(res)
    res.columns = ['Text']
    return res

## TESTING

In [7]:
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
BEARER_TOKEN = ''
ACCESS_TOKEN = ''
ACCESS_TOKEN_SECRET = ''

In [8]:
# criando conexão a api do twitter
client = connect_twitter(BEARER_TOKEN, CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [9]:
# criando BD para adicionar os dados
con, cur = create_db('db_test')

In [10]:
star = datetime(year=2022, month=2, day=26, hour=12, minute=15, second=0)
end_minutes_time = 3
query = 'sentimento'
aux = extracting_data(client_twitter=client, query=query, start_time=star, end_minutes_time=end_minutes_time)

2022-02-26 12:16:00
2022-02-26 12:17:00
2022-02-26 12:18:00


In [11]:
aux[0]

[<Tweet id=1497545926075994113 text="I patriottici diranno che è bellissimo sentire un presidente incitare alla guerra la popolazione civile, ma secondo me non è bello, e secondo me è una sconfitta globale e di tutti. È lo stesso sentimento con cui l'america da 50 anni costringe all'embargo Cuba,">,
 <Tweet id=1497545867015823362 text='RT @yasreis: não finjo sentimento  oq demonstro eu realmente sinto'>,
 <Tweet id=1497545860267360261 text='RT @Glucio03: Sentimento tão puro como este não há mm'>,
 <Tweet id=1497545818651475971 text='que sentimento ruim sério...'>]

In [12]:
insert_db(cur, con, aux)

In [13]:
df = read_data('db_test')
df.head()

,Text
0,I patriottici diranno che è bellissimo sentire...
1,RT @yasreis: não finjo sentimento oq demonstr...
2,RT @Glucio03: Sentimento tão puro como este nã...
3,que sentimento ruim sério...
4,RT @yasreis: não finjo sentimento oq demonstr...
